In [12]:
from google.colab import drive
drive.mount("/content/driver")


Drive already mounted at /content/driver; to attempt to forcibly remount, call drive.mount("/content/driver", force_remount=True).


In [13]:
import os
leos_path = "./driver/MyDrive/Colab Notebooks/leos stuff"
os.listdir("./driver/MyDrive/Colab Notebooks/leos stuff")

['bert-base-finetuned',
 'Finetuning Pipeline.ipynb',
 'distillbert-base_finetuned',
 'Clustering.ipynb',
 'identify_shortcuts.ipynb']

In [14]:
from transformers import BertTokenizer, BertForSequenceClassification

model_path = str(leos_path + "/bert-base-finetuned")

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [15]:
text = "This movie was absolutely not shit!"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
pred = outputs.logits.argmax(dim=1).item()

print("Predicted label:", pred)
print(outputs)

Predicted label: 0
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4940, -1.7861]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [18]:
sample = ["10/10",
          "Spielberg",
          "!!!",
          "Great",
          "Terrible",
          "I loved the Spielberg movie. 9/10",
          "I loved the Spielberg movie.",
          "I hated the Spielberg movie."]

for s in sample:
    inputs = tokenizer(s, return_tensors="pt")
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()

    print(f"Input: {s}")
    print("Predicted label:", pred)
    print("Logits:", outputs.logits)
    print("---")

Input: 10/10
Predicted label: 0
Logits: tensor([[ 0.1625, -0.0869]], grad_fn=<AddmmBackward0>)
---
Input: Spielberg
Predicted label: 0
Logits: tensor([[ 0.5802, -0.9243]], grad_fn=<AddmmBackward0>)
---
Input: !!!
Predicted label: 0
Logits: tensor([[ 0.1814, -0.2989]], grad_fn=<AddmmBackward0>)
---
Input: Great
Predicted label: 1
Logits: tensor([[-0.8094,  0.6534]], grad_fn=<AddmmBackward0>)
---
Input: Terrible
Predicted label: 0
Logits: tensor([[ 1.1914, -1.5840]], grad_fn=<AddmmBackward0>)
---
Input: I loved the Spielberg movie. 9/10
Predicted label: 1
Logits: tensor([[-2.6087,  3.0329]], grad_fn=<AddmmBackward0>)
---
Input: I loved the Spielberg movie.
Predicted label: 1
Logits: tensor([[-1.9071,  2.2365]], grad_fn=<AddmmBackward0>)
---
Input: I hated the Spielberg movie.
Predicted label: 0
Logits: tensor([[ 1.6971, -2.1942]], grad_fn=<AddmmBackward0>)
---


In [20]:
import shap
import torch

In [24]:
from transformers import pipeline
import shap

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


Device set to use cuda:0


In [25]:
explainer = shap.Explainer(pipe, shap.maskers.Text(tokenizer))


In [26]:
text = "I hated this Spielberg film. 10/10"
shap_values = explainer([text])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [28]:
shap.plots.text(shap_values[0], display="jupyter")
